In [1]:
!wget https://genome-idx.s3.amazonaws.com/hisat/mm10_genome.tar.gz -P data/

--2024-05-23 19:06:42--  https://genome-idx.s3.amazonaws.com/hisat/mm10_genome.tar.gz
Resolving genome-idx.s3.amazonaws.com (genome-idx.s3.amazonaws.com)... 3.5.29.95, 3.5.28.182, 3.5.30.217, ...
Connecting to genome-idx.s3.amazonaws.com (genome-idx.s3.amazonaws.com)|3.5.29.95|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3804366597 (3.5G) [binary/octet-stream]
Saving to: ‘data/mm10_genome.tar.gz’

mm10_genome.tar.gz  100%[===================>]   3.54G  22.5MB/s    in 2m 53s  

2024-05-23 19:09:36 (21.0 MB/s) - ‘data/mm10_genome.tar.gz’ saved [3804366597/3804366597]



In [2]:
!tar -xvf data/mm10_genome.tar.gz

mm10/
mm10/genome.8.ht2
mm10/genome.5.ht2
mm10/make_mm10.sh
mm10/genome.7.ht2
mm10/genome.6.ht2
mm10/genome.4.ht2
mm10/genome.3.ht2
mm10/genome.1.ht2
mm10/genome.2.ht2


In [3]:
!wget https://cloud.biohpc.swmed.edu/index.php/s/oTtGWbWjaxsQ2Ho/download

--2024-05-23 19:20:35--  https://cloud.biohpc.swmed.edu/index.php/s/oTtGWbWjaxsQ2Ho/download
Resolving cloud.biohpc.swmed.edu (cloud.biohpc.swmed.edu)... 129.112.9.92
Connecting to cloud.biohpc.swmed.edu (cloud.biohpc.swmed.edu)|129.112.9.92|:443... failed: Connection timed out.
Retrying.

--2024-05-23 19:22:51--  (try: 2)  https://cloud.biohpc.swmed.edu/index.php/s/oTtGWbWjaxsQ2Ho/download
Connecting to cloud.biohpc.swmed.edu (cloud.biohpc.swmed.edu)|129.112.9.92|:443... failed: Connection timed out.
Retrying.

--2024-05-23 19:25:03--  (try: 3)  https://cloud.biohpc.swmed.edu/index.php/s/oTtGWbWjaxsQ2Ho/download
Connecting to cloud.biohpc.swmed.edu (cloud.biohpc.swmed.edu)|129.112.9.92|:443... failed: Connection timed out.
Retrying.

--2024-05-23 19:27:17--  (try: 4)  https://cloud.biohpc.swmed.edu/index.php/s/oTtGWbWjaxsQ2Ho/download
Connecting to cloud.biohpc.swmed.edu (cloud.biohpc.swmed.edu)|129.112.9.92|:443... ^C


In [8]:
srr = "SRR501033"
import os

In [9]:
def get_fastq(accession, fastq_path):
    results = []
    accessions = accession.split("-")
    for acc in accessions:
        print (f"prefetch acc: {acc}")
        exit_status = os.system(f"prefetch {acc} --output-directory {fastq_path}")
        assert exit_status == 0, f"Non-zero exist_status for file {acc}: {exit_status}"
    sra_files = []
    for dirpath, dirnames, filenames in os.walk(fastq_path):
        sra_files += [os.path.join(dirpath,f) for f in filenames if f.endswith("sra")]
    assert len(sra_files) > 0
    print (sra_files)
    for sra_file in sra_files:
        print (f"Running: fasterq-dump {sra_file} --outdir {fastq_path}")
        exit_status = os.system(f"fasterq-dump {sra_file} --outdir {fastq_path}")
        assert exit_status == 0, f"Non-zero fasterq-dump exist_status for file {sra_file}: {exit_status}"
        has_paired = len([f for f in os.listdir(fastq_path) \
                          if f.endswith(".fastq") and (f.find("_1")!=-1 or f.find("_2")!=-1)])
        if has_paired:
            to_remove = [fastq_path + "/" + f for f in os.listdir(fastq_path) \
                          if f.endswith(".fastq") and (f.find("_1")==-1 and f.find("_2")==-1)]
            if len(to_remove) != 0:
                print (f"Warning! Both paired and unpaired fastq found for {sra_file}")
                print ("Following files will be removed: ",to_remove)
                os.system("rm "+" ".join(to_remove))

In [10]:
%time
get_fastq(srr, "fastq")

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.91 µs
prefetch acc: SRR501033

2024-05-23T16:40:00 prefetch.3.1.0: Current preference is set to retrieve SRA Normalized Format files with full base quality scores.
2024-05-23T16:40:00 prefetch.3.1.0: 1) Downloading 'SRR501033'...
2024-05-23T16:40:00 prefetch.3.1.0: SRA Normalized Format file is being retrieved, if this is different from your preference, it may be due to current file availability.
2024-05-23T16:40:00 prefetch.3.1.0:  Downloading via HTTPS...
2024-05-23T16:42:01 prefetch.3.1.0:  HTTPS download succeed
2024-05-23T16:42:06 prefetch.3.1.0:  'SRR501033' is valid
2024-05-23T16:42:06 prefetch.3.1.0: 1) 'SRR501033' was downloaded successfully
['fastq/SRR501033/SRR501033.sra']
Running: fasterq-dump fastq/SRR501033/SRR501033.sra --outdir fastq


spots read      : 63,982,189
reads read      : 63,982,189
reads written   : 63,982,189


UnboundLocalError: local variable 'to_remove' referenced before assignment

In [11]:
# Running: fasterq-dump fastq/SRR501033/SRR501033.sra --outdir fastq # ~8 min

In [12]:
!wget https://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_mouse/release_M10/gencode.vM10.annotation.gff3.gz -P data

--2024-05-23 20:14:45--  https://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_mouse/release_M10/gencode.vM10.annotation.gff3.gz
Resolving ftp.ebi.ac.uk (ftp.ebi.ac.uk)... 193.62.193.165
Connecting to ftp.ebi.ac.uk (ftp.ebi.ac.uk)|193.62.193.165|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29493796 (28M) [application/x-gzip]
Saving to: ‘data/gencode.vM10.annotation.gff3.gz’

gencode.vM10.annota 100%[===================>]  28.13M  1.06MB/s    in 27s     

2024-05-23 20:15:12 (1.06 MB/s) - ‘data/gencode.vM10.annotation.gff3.gz’ saved [29493796/29493796]



In [ ]:
##### Align ####
# !date; hisat2 -p 16 -x data/mm10/genome -U fastq/SRR501033.fastq | samtools view -b > fastq/SRR501033.bam; date;

# Thu May 23 20:09:43 MSK 2024


# 63982189 reads; of these:
#   63982189 (100.00%) were unpaired; of these:
#     21606569 (33.77%) aligned 0 times
#     36570469 (57.16%) aligned exactly 1 time
#     5805151 (9.07%) aligned >1 times
# 66.23% overall alignment rate
# Thu May 23 20:25:08 MSK 2024


In [ ]:
##### samtools #####
# (base) vsfishman@bio-dna:~/GEO2TPM$ date; samtools sort -@ 16 fastq/SRR501033.bam > fastq/SRR501033.sorted.bam; date
# Thu May 23 20:26:10 MSK 2024
# [bam_sort_core] merging from 16 files and 16 in-memory blocks...
# Thu May 23 20:28:33 MSK 2024


In [ ]:
# (base) vsfishman@bio-dna:~/GEO2TPM$ date; stringtie -e -G data/gencode.vM10.annotation.gff3 fastq/SRR501033.sorted.bam -o fastq/SRR501033.gff; date
# Thu May 23 20:31:13 MSK 2024
# Thu May 23 20:34:04 MSK 2024
